### Clase 2

#### Linear regression

1. Data preparation
2. model selection
3. splitting the data
4. model training and evaluation
5. cross-validation
6. hyperparameter tuning
7. final model evaluation

In [3]:
from IPython.display import display, Markdown

from sklearn.datasets import fetch_california_housing
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split